In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import convert_all as crd

In [11]:
# get all low_dim data (head&spine velocity control)
raw_dir = "../demonstrations/raw/put_block_to_bowl"
data = crd.raw_to_dict(
    raw_dir,
    ["records.json"],
    video_file_names=None,
    flatten_mode="hdf5",
    concatenater={
        "/observations/qpos": (
            "/observations/pos_f",
            "/observations/endpos_f",

        ),
        "/action": (
            "/observations/pos_t",
            "/observations/endpos_t",

        ),
    },
    key_filter=[
        "/observations/eef_pose_f",
        "/observations/eef_pose_t",
        "/observations/eff_f",
        "/observations/eff_t",
        "/observations/vel_f",
        "/observations/vel_t",    
    ],

    #eff_f是力反馈，endpos是末端夹爪位置，pos_f是臂的关节数据，eef是末端关节位姿（x,y,z加四元数）vel_t是电流值测定的力反馈，不太准所以不合适加入
    # name_converter={
    #     "//observations/eef_pose_f_left": "/observations/qpos", #注意格式！！！key中有两个//！
    #     "//observations/eef_pose_f_right": "/action",
    # }, #concatenater中已经包含改名的操作，所以这里不需要了

)
print(data["0"]["/observations/qpos"])
#此时返回的data是一个字典
# data = {
#     "1": {
#         "/observations/qpos": [
#             [0, 1, 2],
#             [3, 4, 5]
#         ]
#     },
#     # 其他键值对...
#}

Data Converting:  32%|███▏      | 16/50 [00:00<00:00, 156.69it/s]

Data Converting: 100%|██████████| 50/50 [00:00<00:00, 144.02it/s]

[[-0.000572213320992887, -0.7661936283111572, 0.7036316394805908, 1.5367742776870728, -0.9657053351402283, -1.576066255569458, 0.02940582184048442], [-0.00019073777366429567, -0.7661936283111572, 0.7036316394805908, 1.5363928079605103, -0.9657053351402283, -1.576066255569458, 0.02940582184048442], [-0.00019073777366429567, -0.7661936283111572, 0.7036316394805908, 1.5314335823059082, -0.9649423956871033, -1.576066255569458, 0.02940582184048442], [-0.000572213320992887, -0.7661936283111572, 0.7036316394805908, 1.5238040685653687, -0.9649423956871033, -1.576066255569458, 0.02940582184048442], [-0.000572213320992887, -0.7661936283111572, 0.7036316394805908, 1.5180819034576416, -0.9649423956871033, -1.576066255569458, 0.02940582184048442], [-0.000572213320992887, -0.7661936283111572, 0.7040131092071533, 1.5135042667388916, -0.9634165167808533, -1.5749218463897705, 0.02940582184048442], [-0.000572213320992887, -0.7661936283111572, 0.7040131092071533, 1.5028228759765625, -0.959983229637146, -

In [12]:
ep_number = len(data)
print(f"Number of episodes: {ep_number}")

Number of episodes: 50


In [13]:
print(list(data["0"].keys()))
print(list(data["1"]["/observations/qpos"][0]))
print(list(data["0"]["/action"][0]))

['/observations/time', '/observations/qpos', '/action']
[-0.000572213320992887, -0.7661936283111572, 0.7032501697540283, 1.5363928079605103, -0.9649423956871033, -1.5764477252960205, 0.033121489278681866]
[-0.00019073777366429567, -0.7661936283111572, 0.7040131092071533, 1.5367742776870728, -0.9649423956871033, -1.576066255569458, 0.00048652418024396445]


In [15]:
import os
import cv2
from concurrent.futures import ThreadPoolExecutor

# merge high_dim data and save
raw_dir
video_names = ["0.avi"]
target_dir = f"../data/hdf5/put_block_to_bowl/"
low_dim_data = data
name_converter = {"0": "/observations/images/0"}
target_namer = lambda i: f"episode_{i}.hdf5"

compresser = crd.Compresser("jpg", [int(cv2.IMWRITE_JPEG_QUALITY), 50], True)

os.makedirs(target_dir, exist_ok=True)

# get max episode length
episode_lens = []
for low_d in low_dim_data.values():
    episode_lens.append(len(list(low_d.values())[0]))
max_pad_lenth = max(episode_lens)

episode_names = list(low_dim_data.keys())
print(f"Max episode length: {max_pad_lenth}")
print(f"All episodes: {episode_names}")
print(f"episode number: {len(episode_names)}")

Max episode length: 200
All episodes: ['24', '13', '12', '44', '47', '19', '4', '40', '1', '8', '38', '46', '16', '10', '39', '23', '41', '43', '14', '5', '18', '9', '17', '28', '35', '45', '49', '11', '31', '48', '22', '0', '7', '6', '27', '34', '21', '29', '32', '2', '36', '33', '15', '25', '20', '37', '42', '30', '26', '3']
episode number: 50


In [16]:
# save all data
futures = []
with ThreadPoolExecutor(max_workers=25) as executor:

    def save_one(index, ep_name):
        crd.merge_video_and_save(
            low_dim_data[ep_name],
            f"{raw_dir}/{ep_name}",
            video_names,
            crd.save_dict_to_hdf5,
            name_converter,
            compresser,
            f"{target_dir}/" + target_namer(index),
            max_pad_lenth,
        )
        data.pop(ep_name)

    for index, ep_name in enumerate(episode_names):
        futures.append(executor.submit(save_one, index, ep_name))
#执行到这里，所有数据已经保存完毕

DEBUG:root:compressed_len_list: [[10631, 10629, 10640, 10646, 10632, 10617, 10648, 10595, 10604, 10603, 10523, 10501, 10514, 10550, 10627, 10634, 10648, 10605, 10575, 10564, 10637, 10697, 10702, 10716, 10737, 10805, 10819, 10844, 10881, 10816, 10699, 10796, 11175, 10967, 10860, 11168, 11032, 11358, 11249, 11398, 11620, 11860, 11976, 11980, 12201, 12233, 12166, 12632, 12542, 12583, 12434, 11923, 12115, 11778, 11690, 11466, 11315, 11219, 11076, 10897, 10789, 10827, 10812, 10816, 10784, 11088, 11069, 11377, 11298, 11588, 11867, 12016, 12137, 12586, 12615, 12837, 12758, 12485, 12599, 12981, 12864, 12656, 12605, 12380, 12079, 12071, 12046, 11604, 11361, 11399, 11424, 11837, 11895, 11760, 11768, 11670, 11632, 11693, 11741, 11763, 12379, 12686, 12788, 13158, 13639, 13530, 13701, 13453, 14250, 14422, 14280, 14263, 14152, 14431, 14391, 14560, 14586, 14700, 14566, 14499, 14585, 14513, 14383, 14969, 14939, 15742, 15626, 15737, 15820, 15849, 15595, 14662, 13785, 13936, 13708, 13689, 13990, 14081, 

In [ ]:
# fix action
import json
import numpy as np

for i in range(49, 50):
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "r") as f:
        data = json.load(f)
        action = data["action"]["spine"]["joint_position"]
        obs = data["observation"]["spine"]["joint_position"]
        # change action
        delta = np.array(obs[1:]) - np.array(obs[:-1])
        # > 0 -> 1, < 0 -> -1
        action = np.sign(delta)
        # 获取不为0的值的index
        index_not0 = np.where(action != 0)
        first_not0 = index_not0[0][0]
        last_not0 = index_not0[0][-1]
        # 从第一个不为0的值开始，到最后一个不为0的值结束，将中间的值设为1
        action[first_not0:last_not0] = 1
        action = action.tolist()
        action.append(action[-1])
        data["action"]["spine"] = {"joint_velocity": action}
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "w") as f:
        json.dump(data, f)

In [ ]:
# rename
import json

for i in range(50):
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "r") as f:
        data = json.load(f)
        data["action"]["head"] = {"joint_velocity": data["action"]["head"]["joint_velocity"]["joint_position"]}
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "w") as f:
        json.dump(data, f)

## Position Control

In [ ]:
# add head and spine to joint_position action
import json

for i in range(58):
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "r") as f:
        data = json.load(f)
        obs_spine = data["observation"]["spine"]["joint_position"]
        obs_head = data["observation"]["head"]["joint_position"]
        data["action"]["head"]["joint_position"] = obs_head[1:] + [obs_head[-1]]
        data["action"]["spine"]["joint_position"] = obs_spine[1:] + [obs_spine[-1]]
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "w") as f:
        json.dump(data, f)

In [ ]:
# get all low_dim data (head&spine position control)
import convert_all as crd
raw_dir = "demonstrations/raw/mmk_pick_grape"
data = crd.raw_to_dict(
    raw_dir,
    ["low_dim.json"],
    video_file_names=None,
    flatten_mode="hdf5",
    concatenater={
        "/observations/qpos": (
            "/observation/arm/left/joint_position",
            "/observation/eef/left/joint_position",
            "/observation/arm/right/joint_position",
            "/observation/eef/right/joint_position",
            "/observation/head/joint_position",
            "/observation/spine/joint_position"
        ),
        "/action": (
            "/action/arm/left/joint_position",
            "/action/eef/left/joint_position",
            "/action/arm/right/joint_position",
            "/action/eef/right/joint_position",
            "/action/head/joint_position",
            "/action/spine/joint_position"
        ),
    },
    key_filter=[
        "/observation/arm/left/joint_velocity",
        "/observation/arm/right/joint_velocity",
        "/observation/arm/left/joint_torque",
        "/observation/arm/right/joint_torque",
        "/action/head/joint_velocity",
        "/action/spine/joint_velocity"
    ],
)

In [ ]:
ep_number = len(data)
print(f"Number of episodes: {ep_number}")

In [ ]:
# 将spine的observation和action[-120000, 0]缩放到[-2pi, 0]
import numpy as np
for i in range(ep_number):
    qpos_obs = np.array(data[str(i)]["/observations/qpos"])
    qpos_act = np.array(data[str(i)]["/action"])
    qpos_obs[:, -1] = 2 * np.pi * qpos_obs[:, -1] / 120000
    qpos_act[:, -1] = 2 * np.pi * qpos_act[:, -1] / 120000
    data[str(i)]["/observations/qpos"] = qpos_obs.tolist()
    data[str(i)]["/action"] = qpos_act.tolist()